#### Importing librairies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

#### Importing and processing incidents

In [2]:
start_incidents = pd.read_csv("start_incidents_2018_01_01_2023_08_28.csv")
start_incidents = start_incidents[['UserName','Timestamp', 'Embedded_text','Emojis','Tweet URL']]
start_incidents.insert(0, 'start/end', 'start')
start_incidents['Timestamp'] = pd.to_datetime(start_incidents['Timestamp']) + datetime.timedelta(hours=1) # Add one hour to Timestamp in order to convert UTC to Paris time
start_incidents.head()

,start/end,UserName,Timestamp,Embedded_text,Emojis,Tweet URL
0,start,@Ligne9_RATP,2018-01-01 02:56:08+00:00,"02:53, la rame stationne à Nation en dir. de M...",NaN,https://twitter.com/Ligne9_RATP/status/9476475...
1,start,@Ligne9_RATP,2018-01-01 19:14:08+00:00,"19:11, la rame stationne à Cx Chavaux en dir. ...",NaN,https://twitter.com/Ligne9_RATP/status/9478937...
2,start,@Ligne9_RATP,2018-01-02 12:06:09+00:00,"12:02, le trafic est interrompu entre Rue des ...",NaN,https://twitter.com/Ligne9_RATP/status/9481484...
3,start,@Ligne9_RATP,2018-01-02 18:05:08+00:00,"18:03, le trafic est perturbé sur la ligne (in...",NaN,https://twitter.com/Ligne9_RATP/status/9482387...
4,start,@Ligne9_RATP,2018-01-02 23:38:08+00:00,"23:35, le trafic est interrompu entre Trocader...",NaN,https://twitter.com/Ligne9_RATP/status/9483225...


In [3]:
end_incidents = pd.read_csv("end_incidents_2018_01_01_2023_08_28.csv")
end_incidents = end_incidents[['UserName','Timestamp', 'Embedded_text','Emojis','Tweet URL']]
end_incidents.insert(0, 'start/end', 'end')
end_incidents['Timestamp'] = pd.to_datetime(end_incidents['Timestamp']) + datetime.timedelta(hours=1) # Add one hour to Timestamp in order to convert UTC to Paris time
end_incidents.head()

,start/end,UserName,Timestamp,Embedded_text,Emojis,Tweet URL
0,end,@Ligne9_RATP,2018-01-01 02:22:11+00:00,"Incident terminé (personne sur les voies), ret...",NaN,https://twitter.com/Ligne9_RATP/status/9476390...
1,end,@Ligne9_RATP,2018-01-01 02:46:17+00:00,Retour à un trafic régulier sur l'ensemble de ...,NaN,https://twitter.com/Ligne9_RATP/status/9476451...
2,end,@Ligne9_RATP,2018-01-02 12:11:08+00:00,"12:09, le trafic reprend progressivement (pann...",NaN,https://twitter.com/Ligne9_RATP/status/9481496...
3,end,@Ligne9_RATP,2018-01-02 12:24:06+00:00,Retour à un trafic régulier sur l'ensemble de ...,NaN,https://twitter.com/Ligne9_RATP/status/9481529...
4,end,@Ligne9_RATP,2018-01-02 18:26:07+00:00,Incident terminé.Retour à un trafic normal sur...,NaN,https://twitter.com/Ligne9_RATP/status/9482440...


#### Merging start and end incidents

In [4]:
incidents = pd.concat([start_incidents, end_incidents])
incidents["Timestamp"] = incidents["Timestamp"].apply(lambda x: str(x))
incidents.sort_values(by=['Timestamp'], inplace=True)
incidents.reset_index(inplace=True)
incidents.drop(columns=['index'], inplace=True)
incidents["date"] = incidents["Timestamp"].apply(lambda x: x.split(" ")[0])
incidents.head()

,start/end,UserName,Timestamp,Embedded_text,Emojis,Tweet URL,date
0,end,@Ligne9_RATP,2018-01-01 02:22:11+00:00,"Incident terminé (personne sur les voies), ret...",NaN,https://twitter.com/Ligne9_RATP/status/9476390...,2018-01-01
1,end,@Ligne9_RATP,2018-01-01 02:46:17+00:00,Retour à un trafic régulier sur l'ensemble de ...,NaN,https://twitter.com/Ligne9_RATP/status/9476451...,2018-01-01
2,start,@Ligne9_RATP,2018-01-01 02:56:08+00:00,"02:53, la rame stationne à Nation en dir. de M...",NaN,https://twitter.com/Ligne9_RATP/status/9476475...,2018-01-01
3,start,@Ligne9_RATP,2018-01-01 19:14:08+00:00,"19:11, la rame stationne à Cx Chavaux en dir. ...",NaN,https://twitter.com/Ligne9_RATP/status/9478937...,2018-01-01
4,start,@Ligne9_RATP,2018-01-02 12:06:09+00:00,"12:02, le trafic est interrompu entre Rue des ...",NaN,https://twitter.com/Ligne9_RATP/status/9481484...,2018-01-02


In [5]:
def timestamp_to_time_slot(timestamp : str) :
    """
    Return the time slot of a given timestamp.
    """
    time_slot = int(timestamp[11:13])//2

    time_slots = ["0h-2h","2h-4h","4h-6h","6h-8h","8h-10h","10h-12h","12h-14h","14h-16h","16h-18h","18h-20h","20h-22h","22h-24h"]
    time_slots_dict = {i : time_slots[i] for i in range(len(time_slots))}    
    
    return time_slots_dict[time_slot]

In [6]:
incidents["time_slot"] = incidents["Timestamp"].apply(timestamp_to_time_slot)
incidents.head()

,start/end,UserName,Timestamp,Embedded_text,Emojis,Tweet URL,date,time_slot
0,end,@Ligne9_RATP,2018-01-01 02:22:11+00:00,"Incident terminé (personne sur les voies), ret...",NaN,https://twitter.com/Ligne9_RATP/status/9476390...,2018-01-01,2h-4h
1,end,@Ligne9_RATP,2018-01-01 02:46:17+00:00,Retour à un trafic régulier sur l'ensemble de ...,NaN,https://twitter.com/Ligne9_RATP/status/9476451...,2018-01-01,2h-4h
2,start,@Ligne9_RATP,2018-01-01 02:56:08+00:00,"02:53, la rame stationne à Nation en dir. de M...",NaN,https://twitter.com/Ligne9_RATP/status/9476475...,2018-01-01,2h-4h
3,start,@Ligne9_RATP,2018-01-01 19:14:08+00:00,"19:11, la rame stationne à Cx Chavaux en dir. ...",NaN,https://twitter.com/Ligne9_RATP/status/9478937...,2018-01-01,18h-20h
4,start,@Ligne9_RATP,2018-01-02 12:06:09+00:00,"12:02, le trafic est interrompu entre Rue des ...",NaN,https://twitter.com/Ligne9_RATP/status/9481484...,2018-01-02,12h-14h


#### Initialise negatif_df

In [7]:
columns = ["month_number","week_number","monthday_number","weekday_number","time_slot","ongoing_incident"]

In [8]:
# Pourrait être remplacé par : pandas.date_range() ???

def generate_time_stamps(start_day : str, end_day: str,time_ressolution = 2) :
    """
    Generate a list of time stamps between <start_day> and <end_day> (all day include) with a given time resolution.
    """
    start_day = datetime.datetime.strptime(start_day, '%Y-%m-%d')
    end_day = datetime.datetime.strptime(end_day, '%Y-%m-%d') + datetime.timedelta(days=1)
    time_stamps = []
    
    time_stamp = start_day

    while time_stamp < end_day :
        time_stamps.append(time_stamp)
        time_stamp += datetime.timedelta(hours=time_ressolution)


    return time_stamps

time_stamps = [datetime.datetime.strftime(time_stamp, '%Y-%m-%d %H:%M:%S') for time_stamp in generate_time_stamps("2018-01-01", "2023-08-28")]

In [9]:
negatif_df = pd.DataFrame(time_stamps, columns=["timestamp"])
negatif_df["month_number"] = negatif_df["timestamp"].apply(lambda x : datetime.datetime.strptime(x[:10], '%Y-%m-%d').month)
negatif_df["week_number"] = negatif_df["timestamp"].apply(lambda x : datetime.datetime.strptime(x[:10], '%Y-%m-%d').isocalendar().week)
negatif_df["monthday_number"] = negatif_df["timestamp"].apply(lambda x : datetime.datetime.strptime(x[:10], '%Y-%m-%d').day)
negatif_df["weekday_number"] = negatif_df["timestamp"].apply(lambda x : datetime.datetime.strptime(x[:10], '%Y-%m-%d').weekday())
negatif_df["time_slot"] = negatif_df["timestamp"].apply(timestamp_to_time_slot)
negatif_df.insert(1, 'date', negatif_df["timestamp"].apply(lambda x : x[:10]))
negatif_df["ongoing_incident"] = False

negatif_df.head()

,timestamp,date,month_number,week_number,monthday_number,weekday_number,time_slot,ongoing_incident
0,2018-01-01 00:00:00,2018-01-01,1,1,1,0,0h-2h,False
1,2018-01-01 02:00:00,2018-01-01,1,1,1,0,2h-4h,False
2,2018-01-01 04:00:00,2018-01-01,1,1,1,0,4h-6h,False
3,2018-01-01 06:00:00,2018-01-01,1,1,1,0,6h-8h,False
4,2018-01-01 08:00:00,2018-01-01,1,1,1,0,8h-10h,False


#### Pass line with incident to True in negatif_df

In [10]:
complet_df = pd.merge(negatif_df, incidents[["date","time_slot","Embedded_text", "Tweet URL"]], how="left", on=["date","time_slot"])
complet_df.drop_duplicates(subset=["timestamp","time_slot"], inplace=True, keep="first")
complet_df["ongoing_incident"] = complet_df["Embedded_text"].apply(lambda x : False if pd.isna(x) else True) # if "Embedded_text" is not NaN, set "ongoing_incident" to True in coreesponding row
complet_df.head()# ATTENTION : il peut y avoir plusieurs lignes pour un même incident et pour un même time_slot
complet_df.shape

(24792, 10)

#### Merging complet_df with holidays and public holiday

In [11]:
holiday_df = pd.read_csv("school_holiday_calendar.csv")
complet_df = complet_df.merge(holiday_df, on = "date", how = "left")

In [12]:
public_holidays = pd.read_csv("jours_feries_metropole.csv")
public_holidays = public_holidays[["date","nom_jour_ferie"]]
public_holidays["date"] = pd.to_datetime(public_holidays["date"], format="%Y-%m-%d")
public_holidays.rename(columns={"nom_jour_ferie":"public_holiday"}, inplace=True)
public_holidays["public_holiday"] = True
public_holidays["date"] = public_holidays["date"].astype(object)
public_holidays.fillna(False, inplace=True)
complet_df = complet_df.merge(public_holidays, on = "date", how = "left", )

Fill NaN

In [13]:
columns = public_holidays.columns.values.tolist() + complet_df.columns.values.tolist()
columns = list(set(columns))
columns.remove("date")
complet_df[columns] = complet_df[columns].fillna(value=False)

In [14]:
#Convertire pd.parse_date